In [156]:
import numpy as np
import scipy
import pandas as pd

In [157]:
def sowftmax(beta, q):
    return (np.exp(q * beta)) / sum(np.exp(q * beta))

def prediction_error(a , b): return a-b

def update_q_table(q_table,action,reward,alpha):
    q_table[action] = q_table[action] + (alpha * (prediction_error(reward,q_table[action])))


In [162]:
# set parameters

# set 1 to pritn every trail 
PRINT_EACH_TRAIL = 0

# set num of trails
num_of_trails = 2000

# set num of action 
action_space = 2

# create the q_table 
q_table = np.zeros(action_space)

# set Learning Rate -  α  
alpha = 0.2

# set Discounting Factor -  γ 
gamma = 1

# set beta - temperature - β (sowftMax)
beta = 5

# probabilty for a reward for each action in stage 2
prob_reward = np.array([[.9,.1],
                        [.1,.9]])

reward_list = np.zeros(num_of_trails)

action_list = np.zeros(num_of_trails)


In [163]:
for t in range(num_of_trails):
    
    if PRINT_EACH_TRAIL: print("----- Trail = {} -----".format(t))
    
    # calc sowftMax
    prob = sowftmax(beta,q_table)
    if PRINT_EACH_TRAIL: print("Prob = {}".format(prob))
    
    # choose an action according to sowftMax
    action = np.random.choice([0,1] , p = prob)
    action_list[t] = action
    if PRINT_EACH_TRAIL: print("Action = {}".format(action))
    
    # cheek if the trail is rewarded according to probs 
    reward = np.random.choice([0,1] , p = prob_reward[0]) if action == 0 \
                                                    else np.random.choice([0,1] , p = prob_reward[1])
    
    reward_list[t] = reward
    if PRINT_EACH_TRAIL: print("Reward = {}".format(reward))
    
    # update q_table according to q_learning formula
    update_q_table(q_table,action,reward,alpha)
    if PRINT_EACH_TRAIL: print("Q = {}".format(q_table.tolist()))
    if PRINT_EACH_TRAIL : input('PRESS ENTER FOR NEXT TRAIL')


In [164]:
data = {'action' :action_list ,  'reward' : reward_list }
df = pd.DataFrame(data,columns = ['action' ,'reward'])
print("The avarge rewared is = {}".format(df['reward'].mean()))

print(df.head(10))


The avarge rewared is = 0.884
   action  reward
0     0.0     1.0
1     0.0     1.0
2     0.0     0.0
3     0.0     0.0
4     0.0     0.0
5     0.0     0.0
6     0.0     0.0
7     0.0     0.0
8     0.0     0.0
9     1.0     1.0


In [165]:
def param_recovary_f(paramters, df):
    
    ll = 0 
    q_table = np.zeros(2)

    num_trials = len(df)
    action = list(map(int, df['action']))
    reward = list(map(int, df['reward'])) 
    p_choice= np.zeros(num_trials)
    
    beta = paramters[0]
    alpha = paramters[1]

    for t in range(num_trials):
        p_choice[t] = np.exp(beta * q_table[action[t]]) / sum(np.exp(beta*q_table))
        p_e = reward[t] - q_table[action[t]]
        q_table[action[t]] = q_table[action[t]] + alpha*p_e

    ll = -(sum(np.log(p_choice)))
    return ll

def param_recovary(param):
    return param_recovary_f(param,df)

minimize(param_recovary,[1,0.5],method = 'Nelder-Mead')

<ipython-input-165-b5c73cb435a4>:19: RuntimeWarning: divide by zero encountered in log
  ll = -(sum(np.log(p_choice)))


 final_simplex: (array([[4.87011762, 0.19886181],
       [4.87016486, 0.19888083],
       [4.87008029, 0.19889725]]), array([172.08895765, 172.08895765, 172.08895774]))
           fun: 172.08895764533668
       message: 'Optimization terminated successfully.'
          nfev: 105
           nit: 54
        status: 0
       success: True
             x: array([4.87011762, 0.19886181])